<a href="https://colab.research.google.com/github/BhuvaneswariGudivaka/FMML-LAB-1/blob/main/Mod3_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FOUNDATIONS OF MODERN MACHINE LEARNING, IIIT Hyderabad**
### MODULE: CLASSIFICATION-1
### LAB-3 : Using KNN for Text Classification
#### Module Coordinator: Jashn Arora


---

## **Section 1: Understanding NLP tools**

In this lab we will be using KNN on a real world NLP application i.e. is text classification. But first look at some NLP techniques for text classification and tools that we use when we want to use python for NLP.

## Section 1.2: Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.  
In case of text, there are lots of things that need to be taken into account.  


1.   Removing numbers from the text
2.   Handling capitalization and punctuation.
3.   Stemming and Lemmatizing text.  

And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.



### **NLTK**
NLTK (or Natural Language Tool Kit) is a commonly used library for processing text. We will use this tool in this lab. Lets first install it.


In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [3]:
sample_text = "Troubling"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

Troubling
troubl
trouble


## Section 1.2: BAG OF WORDS

A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document.
It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [4]:
5*12

60

In [5]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()
    return bag_of_words_train, bag_of_words_test


## Section 1.3: TF-IDF
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

 Document frequency is the number of documents in which the word is present.  Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term *t*.




In [6]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

# **Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET**

Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.  
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).

In [ ]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
df.to_csv('reviews.csv', index=False)

# **Section 3: KNN MODEL**

Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF.
Note the different metrics and parameters used in each.

In [ ]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 1: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2,
                                         metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

Note: Cross-validation will be discussed in detail in the upcoming lab session.

In [ ]:
## KNN accuracy after using BoW
predicted, y_test = bow_knn()

In [ ]:
## KNN accuracy after using TFIDF
predicted, y_test = tfidf_knn()

# Section 4: SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [ ]:
# Upload the spam text data CSV file that has been shared with you. You can also download the file from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv('spam.csv', error_bad_lines=False)
df

In [ ]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [ ]:
df.head(5)

In [ ]:
len(df)

In [ ]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [ ]:
# This cell may take some time to run
predicted, y_test = bow_knn()

In [ ]:
# This cell may take some time to run
predicted, y_test = tfidf_knn()

### Questions to Think About and Answer
1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
2. Can you think of techniques that are better than both BoW and TF-IDF ?
3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.

#QUESTION 1
The TF-IDF (Term Frequency-Inverse Document Frequency) approach generally results in better accuracy than the Bag-of-Words (BoW) model for several reasons:

Term Importance: TF-IDF considers not only the frequency of terms in a document (as in BoW) but also their importance in the entire corpus. Terms that are frequent in a document but rare in the corpus are weighted higher, indicating their significance in representing the content of the document.

Normalization: TF-IDF normalizes term frequencies by their inverse document frequency, reducing the impact of common terms that occur frequently across all documents. This helps in focusing on terms that are more discriminative and informative for distinguishing between documents.

Reduction of Noise: By giving lower weight to common terms that occur in many documents (e.g., "the", "and"), TF-IDF reduces the noise introduced by such terms in the representation of documents, leading to more effective feature representations.

Dimensionality Reduction: TF-IDF implicitly performs dimensionality reduction by weighting terms based on their importance. This can help in reducing the dimensionality of the feature space compared to BoW, making the model less prone to overfitting and improving generalization performance.

Semantic Similarity: TF-IDF can capture semantic similarity between documents better than BoW. Terms that are semantically related but not exact matches may still receive high TF-IDF scores if they co-occur frequently within a document and are rare in other documents.

Better Representation: Overall, TF-IDF provides a richer and more nuanced representation of documents compared to BoW by taking into account both term frequency and their significance in the corpus. This leads to more accurate and informative feature representations for downstream tasks such as text classification, clustering, or retrieval.

While TF-IDF generally outperforms BoW, its effectiveness may still depend on factors such as the nature of the dataset, the task at hand, and the specific implementation details. It's important to experiment and validate different text representation methods to determine the most suitable approach for a given problem.






#QUESTION 2
Certainly! There are several techniques that have been developed to improve upon the Bag-of-Words (BoW) and TF-IDF approaches for text representation. Some of these techniques include:

Word Embeddings:

Word embeddings such as Word2Vec, GloVe, and FastText represent words as dense vectors in a continuous vector space, capturing semantic relationships between words.
Unlike BoW and TF-IDF, word embeddings consider the context of words and capture semantic similarity, allowing for more nuanced representations of text.
Doc2Vec:

Doc2Vec extends the idea of word embeddings to capture document-level semantics by representing entire documents as dense vectors.
It learns distributed representations of documents in an unsupervised manner, enabling tasks such as document similarity, classification, and clustering.
BERT (Bidirectional Encoder Representations from Transformers):

BERT is a transformer-based model that learns contextualized representations of words and sentences by training on large amounts of text data.
It captures bidirectional context from both left and right contexts, allowing for more accurate representations of words and sentences compared to traditional word embeddings.
ELMo (Embeddings from Language Models):

ELMo generates contextualized word embeddings by leveraging the internal states of a bidirectional LSTM language model.
It captures word meanings in context by considering the surrounding words, leading to more informative representations compared to static word embeddings.
Transformer-Based Models:

Transformer-based models like GPT (Generative Pre-trained Transformer) and GPT-2 are pre-trained language models that generate contextualized representations of text.
These models can be fine-tuned on specific tasks or used to extract features for downstream tasks such as text classification, sentiment analysis, and named entity recognition.
Attention Mechanisms:

Attention mechanisms, commonly used in transformer architectures, allow models to focus on relevant parts of the input text when generating representations.
By dynamically attending to different parts of the input sequence, attention mechanisms can capture long-range dependencies and improve the quality of representations.
These techniques offer more sophisticated ways of representing text compared to BoW and TF-IDF by capturing contextual information, semantic relationships, and long-range dependencies in text data. They have been shown to achieve state-of-the-art performance on various natural language processing tasks and are widely used in both research and industry applications.




#QUESTION 3
Stemming and lemmatization are both techniques used in natural language processing to reduce words to their base or root forms. While they serve a similar purpose, they have different approaches and trade-offs. Here's a comparison of the pros and cons of each:

Stemming:

Pros:
Simplicity: Stemming algorithms are relatively simple and computationally efficient, making them easy to implement and fast to execute.
Speed: Stemming is generally faster than lemmatization since it involves applying a set of rules or heuristics to truncate words to their stems.
Reduction of Variance: Stemming reduces word variations to their common stems, which can help in reducing the dimensionality of text data and improving the efficiency of downstream tasks such as text classification or clustering.
Cons:
Over-Stemming: Stemming algorithms may produce overly aggressive truncations, leading to the loss of meaning or ambiguity in the resulting stems. For example, "running" and "runner" may both be stemmed to "run," losing the distinction between different word forms.
Inaccuracy: Stemming algorithms rely on heuristic rules and pattern matching, which may not always accurately capture the intended word stem. This can lead to incorrect stemming and loss of information.
Language Dependency: Stemming algorithms are language-dependent and may not perform equally well across different languages or language varieties. Some languages may have complex morphological structures that are not well-handled by standard stemming algorithms.
Lemmatization:

Pros:
Precision: Lemmatization produces accurate base forms of words by considering their morphological analysis and dictionary lookup. This ensures that the resulting lemmas are valid words with preserved meaning.
Context Preservation: Lemmatization preserves the semantic meaning of words by mapping them to their canonical forms, reducing the risk of ambiguity or loss of information.
Language Flexibility: Lemmatization algorithms can be tailored to specific languages or domains by using language-specific dictionaries or rule sets, allowing for more accurate and adaptable processing.
Cons:
Complexity: Lemmatization algorithms are more complex and computationally intensive compared to stemming, as they often involve part-of-speech tagging, morphological analysis, and dictionary lookups.
Speed: Lemmatization is generally slower than stemming due to the additional linguistic processing required to determine the base forms of words.
Resource Requirements: Lemmatization may require additional linguistic resources such as dictionaries, part-of-speech taggers, or morphological analyzers, which may not be readily available for all languages or domains.
In summary, both stemming and lemmatization have their advantages and disadvantages. Stemming is simple and fast but may sacrifice precision and accuracy. Lemmatization, on the other hand, produces more accurate results but requires more computational resources and linguistic analysis. The choice between the two techniques depends on the specific requirements of the NLP task, the desired trade-offs between simplicity and accuracy, and the available resources.




### Useful Resources for further reading
1. Stemming and Lemmatization: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
2. TF-IDF and BoW : https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
3. TF-IDF: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
